In [1]:
import pdfplumber
import pandas as pd
import numpy as np
import os

Algoritmo baseado no tutorial de "https://kavita-ganesan.com/extracting-keywords-from-text-tfidf/#.YOcd8XVKgUS"

In [2]:
def extract_text(filename):
    raw_text = list()
    with pdfplumber.open("res/" + filename) as pdf:
        for pdf_page in pdf.pages:
            single_page = pdf_page.extract_text()
            if single_page:
                raw_text.append(single_page)
    
        for i, text in enumerate(raw_text):
            position = raw_text[i].find("Resumo")
            if position > 0:
                raw_text = raw_text[i:]
                return raw_text

In [3]:
import re
import string

def remove_punctuation(raw_text):
    raw_text = re.sub("[%s]" % re.escape(string.punctuation), "", raw_text)
    raw_text = re.sub("\w*\d\w*", "", raw_text)
    raw_text = ' '.join([w for w in raw_text.split() if len(w) < 12])
    raw_text.strip()
    return raw_text

In [4]:
filenames = ["4.pdf", "2.pdf", "3.pdf", "5.pdf", "1.pdf"]
documents_list = list()

for f in filenames:
    raw_text = extract_text(f)
    raw_text = " ".join(raw_text)
    raw_text = raw_text.replace("\n", " ")
    clean_text = remove_punctuation(raw_text)
    documents_list.append(clean_text)

df = pd.DataFrame({"nome arquivos": filenames[0:4], "lista de palavras":documents_list[0:4]})
df
df_test = pd.DataFrame({"nome arquivo": filenames[4:], "lista de palavras": documents_list[4:]})
df_test

,nome arquivo,lista de palavras
0,1.pdf,Redes neurais para previsa˜o do coeﬁciente de ...


In [5]:
docs = df['lista de palavras'].tolist()
docs_test = df_test['lista de palavras'].tolist()

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words
from sklearn.feature_extraction.text import CountVectorizer
sw = stopwords.words("portuguese")

cv = CountVectorizer(max_df=0.85,stop_words=sw)
word_count_vector = cv.fit_transform(docs)

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf.fit(word_count_vector)

TfidfTransformer()

In [8]:
feature_names = cv.get_feature_names() 
doc = docs_test[0]
tf_idf_vector = tfidf.transform(cv.transform([doc]))

coo_matrix = tf_idf_vector.tocoo()

In [9]:
tuples = zip(coo_matrix.col, coo_matrix.data)
sorted_items = sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

sorted_items = sorted_items[:5]

for idx, score in sorted_items:
    print(feature_names[idx])

rede
treinamento
sae
uniforme
valores
